# Symbolic Reasoning & Constraint Checking

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/neurosym-kg/blob/main/notebooks/05_symbolic_reasoning.ipynb)

This notebook explores the symbolic reasoning capabilities of NeuroSym-KG:

- **Rule Engine**: Forward-chaining inference with custom rules
- **Constraint Checker**: Verify LLM outputs against KG facts
- **Path Validator**: Ensure reasoning paths are valid

These tools help ensure faithfulness and reduce hallucinations.

## 1. Setup

In [ ]:
!pip install -q pydantic pydantic-settings httpx tenacity networkx numpy tqdm

In [ ]:
from neurosym_kg import InMemoryKG, Triple
from neurosym_kg.symbolic import (
    RuleEngine,
    Rule,
    TriplePattern,
    ConstraintChecker,
    PathValidator,
    TRANSITIVITY_RULES,
    INHERITANCE_RULES,
    INVERSE_RULES,
)

print("✅ Imports successful!")

## 2. Rule Engine Basics

The rule engine allows you to define logical rules and infer new facts.

In [ ]:
# Create rule engine
engine = RuleEngine()

# Define a simple rule: grandparent
# IF (X, parent_of, Y) AND (Y, parent_of, Z) THEN (X, grandparent_of, Z)
grandparent_rule = Rule(
    name="grandparent",
    antecedent=[
        TriplePattern("?x", "parent_of", "?y"),
        TriplePattern("?y", "parent_of", "?z"),
    ],
    consequent=TriplePattern("?x", "grandparent_of", "?z"),
    description="If X is parent of Y and Y is parent of Z, then X is grandparent of Z",
)

engine.add_rule(grandparent_rule)
print(f"Rule: {grandparent_rule.to_text()}")

In [ ]:
# Create some facts
facts = [
    Triple("Alice", "parent_of", "Bob"),
    Triple("Bob", "parent_of", "Charlie"),
    Triple("Bob", "parent_of", "Diana"),
    Triple("Charlie", "parent_of", "Eve"),
]

print("📚 Initial Facts:")
for f in facts:
    print(f"  {f.to_text()}")

In [ ]:
# Apply forward chaining to infer new facts
inferred = engine.forward_chain(facts)

print("\n✨ Inferred Facts:")
for f in inferred:
    print(f"  {f.to_text()}")

print(f"\nTotal inferred: {len(inferred)} new facts")

## 3. Built-in Rule Sets

NeuroSym-KG includes predefined rules for common patterns.

In [ ]:
# Create engine with transitivity rules
trans_engine = RuleEngine()
trans_engine.add_rules(TRANSITIVITY_RULES)

print("🔄 Transitivity Rules:")
for rule in TRANSITIVITY_RULES:
    print(f"  • {rule.name}: {rule.description}")

In [ ]:
# Test transitivity with location facts
location_facts = [
    Triple("Eiffel_Tower", "located_in", "Paris"),
    Triple("Paris", "located_in", "France"),
    Triple("France", "located_in", "Europe"),
]

print("📍 Location Facts:")
for f in location_facts:
    print(f"  {f.to_text()}")

# Infer transitive closure
inferred_locations = trans_engine.forward_chain(location_facts)

print("\n✨ Inferred Locations:")
for f in inferred_locations:
    print(f"  {f.to_text()}")

In [ ]:
# Inverse rules example
inv_engine = RuleEngine()
inv_engine.add_rules(INVERSE_RULES)

print("↔️ Inverse Rules:")
for rule in INVERSE_RULES:
    print(f"  • {rule.name}: {rule.description}")

# Test with family facts
family_facts = [
    Triple("Alice", "parent_of", "Bob"),
    Triple("Carol", "spouse", "Dave"),
]

inferred_family = inv_engine.forward_chain(family_facts)
print("\n✨ Inferred Relationships:")
for f in inferred_family:
    print(f"  {f.to_text()}")

## 4. Pattern Matching & Queries

In [ ]:
# Query facts using patterns
all_facts = location_facts + inferred_locations

# Find all things located in Europe
pattern = TriplePattern("?thing", "located_in", "Europe")
print(f"Query: {pattern.to_text()}")

results = trans_engine.query(pattern, all_facts)

print("\nResults:")
for binding in results:
    print(f"  ?thing = {binding['?thing']}")

In [ ]:
# Complex query: Find all location relationships
pattern2 = TriplePattern("?x", "located_in", "?y")
results2 = trans_engine.query(pattern2, all_facts)

print("All location relationships:")
for binding in results2:
    print(f"  {binding['?x']} is located in {binding['?y']}")

## 5. Constraint Checking

Verify LLM answers against KG facts to reduce hallucinations.

In [ ]:
# Create a knowledge graph
kg = InMemoryKG()
kg.add_triples([
    Triple("Paris", "capital_of", "France"),
    Triple("Paris", "type", "City"),
    Triple("France", "type", "Country"),
    Triple("Eiffel_Tower", "located_in", "Paris"),
    Triple("Christopher_Nolan", "director_of", "Inception"),
    Triple("Christopher_Nolan", "born_in", "London"),
])

# Create constraint checker
checker = ConstraintChecker(kg, strict_mode=False)

print("✅ Constraint checker initialized")
print(f"   KG has {kg.num_triples} triples")

In [ ]:
# Verify a correct answer
result = checker.verify_answer(
    answer="Paris",
    question="What is the capital of France?",
    reasoning_paths=[[Triple("Paris", "capital_of", "France")]],
)

print("Verification for 'Paris':")
print(f"  Valid: {result.is_valid}")
print(f"  Verified facts: {len(result.verified_facts)}")
print(f"  Violations: {len(result.violations)}")
print(f"  Confidence adjustment: {result.confidence_adjustment:+.2f}")

In [ ]:
# Verify a potentially wrong answer
result2 = checker.verify_answer(
    answer="Berlin",  # Wrong!
    question="What is the capital of France?",
    reasoning_paths=[[Triple("Berlin", "capital_of", "France")]],  # This triple doesn't exist
)

print("Verification for 'Berlin':")
print(f"  Valid: {result2.is_valid}")
print(f"  Violations: {len(result2.violations)}")

for v in result2.violations:
    print(f"    ⚠️ [{v.constraint_type.value}] {v.message}")

In [ ]:
# Verify with type constraints
result3 = checker.verify_answer(
    answer="France",
    question="Where is the Eiffel Tower?",
    expected_type="City",  # France is a Country, not a City
)

print("Verification for 'France' (expected City):")
print(f"  Valid: {result3.is_valid}")
for v in result3.violations:
    print(f"    ⚠️ [{v.severity}] {v.message}")

## 6. Custom Constraints

In [ ]:
from neurosym_kg.symbolic import ConstraintViolation, ConstraintType

# Define custom constraint: answers shouldn't be numbers
def no_numeric_answers(answer: str, question: str, paths) -> list:
    violations = []
    if answer.isdigit():
        violations.append(ConstraintViolation(
            constraint_type=ConstraintType.CUSTOM,
            message=f"Answer '{answer}' is purely numeric",
            severity="warning",
        ))
    return violations

# Add custom constraint
checker.add_custom_constraint(no_numeric_answers)

# Test
result4 = checker.verify_answer(
    answer="2010",
    question="When was Inception released?",
)

print("Verification for '2010':")
for v in result4.violations:
    print(f"  ⚠️ {v.message}")

## 7. Path Validation

In [ ]:
# Create path validator
validator = PathValidator(kg)

# Valid path: Connected triples
valid_path = [
    Triple("Inception", "director", "Christopher_Nolan"),
    Triple("Christopher_Nolan", "born_in", "London"),
]

is_valid, issues = validator.validate_path(valid_path)
print(f"Valid path: {is_valid}")
if issues:
    print(f"  Issues: {issues}")

In [ ]:
# Invalid path: Disconnected triples
invalid_path = [
    Triple("Inception", "director", "Christopher_Nolan"),
    Triple("Paris", "capital_of", "France"),  # Not connected!
]

is_valid2, issues2 = validator.validate_path(invalid_path)
print(f"Disconnected path: {is_valid2}")
for issue in issues2:
    print(f"  ⚠️ {issue}")

In [ ]:
# Path with cycle
cyclic_path = [
    Triple("A", "rel", "B"),
    Triple("B", "rel", "C"),
    Triple("C", "rel", "A"),  # Cycle back to A
    Triple("A", "rel", "D"),
]

is_valid3, issues3 = validator.validate_path(cyclic_path)
print(f"Cyclic path valid: {is_valid3}")
for issue in issues3:
    print(f"  ⚠️ {issue}")

## 8. Integrating with Reasoners

Use constraint checking to post-process reasoner outputs.

In [ ]:
from neurosym_kg import MockLLMBackend, ThinkOnGraphReasoner

# Set up reasoner
llm = MockLLMBackend()
llm.add_response(r".*", "Paris")  # Simple mock

kg_for_reasoner = InMemoryKG()
kg_for_reasoner.add_triples([
    Triple("Paris", "capital_of", "France"),
    Triple("France", "type", "Country"),
])

reasoner = ThinkOnGraphReasoner(kg=kg_for_reasoner, llm=llm, max_depth=1)
checker = ConstraintChecker(kg_for_reasoner)

# Reason and verify
result = reasoner.reason("What is the capital of France?")

# Verify the answer
updated_result, verification = checker.verify_reasoning_result(
    result,
    question="What is the capital of France?",
)

print("📝 Reasoning Result:")
print(f"   Answer: {updated_result.answer}")
print(f"   Original confidence: {result.confidence:.2f}")
print(f"   Verified confidence: {updated_result.confidence:.2f}")
print(f"   Verification: {verification.explanation}")

## 🎯 Summary

This notebook covered:

1. **Rule Engine**: Define and apply logical rules for inference
2. **Built-in Rules**: Transitivity, inheritance, inverse relations
3. **Pattern Matching**: Query facts using variable patterns
4. **Constraint Checking**: Verify answers against KG facts
5. **Custom Constraints**: Add domain-specific validation rules
6. **Path Validation**: Ensure reasoning paths are logically connected

### Benefits

- **Reduced Hallucinations**: Verify LLM outputs against ground truth
- **Explainability**: Show which facts support the answer
- **Confidence Calibration**: Adjust confidence based on verification
- **Domain Rules**: Encode expert knowledge as rules